# Lab | Case Regression

<br>

<details><summary>▶ Instructions:</summary>
<p>

At this point, we have created a model to predict who will make a donation and who won't. But, what about the ammount of money that each person will give? In this lab, subset those that made a donation and use that subset to create a model to predict how much money will they give.

Evaluate the result of your model and estimate how much better the result are for the bussiness in comparison with the naive scenario we discuss on Monday.

</p>
</details>

In [1]:
import pandas as pd  # the same as the lab before
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import resample
from matplotlib import cm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt
import warnings
warnings.filterwarnings('ignore')

categorical = pd.read_csv("files_for_lab/categorical.csv")
numerical = pd.read_csv('files_for_lab/numerical.csv')
target = pd.read_csv('files_for_lab/target.csv')
target.head(1)    # b=boolean/response previous mailing and d = dollar/amount_donated as response

,TARGET_B,TARGET_D
0,0,0.0


In [2]:
print('cat:', categorical.shape)
print('num:', numerical.shape)
print('target_B:\n' + str(target['TARGET_B'].value_counts()))

cat: (95412, 22)
num: (95412, 315)
target_B:
0    90569
1     4843
Name: TARGET_B, dtype: int64


In [3]:
data = pd.concat([categorical, numerical, target], axis=1) 

In [4]:
nans = pd.DataFrame(data.isna().sum()*100/len(data), columns=['percentage'])
nans.sort_values('percentage', ascending = False).head()
data[data.isna().any(axis=1)]   # no NaN's at all

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,...,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,TARGET_B,TARGET_D


In [5]:
# select the features of the people who once donated
category_1 = data[data['TARGET_B']== 1 ]

In [6]:
y = category_1['TARGET_D']
X = category_1.drop(['TARGET_B','TARGET_D'], axis = 1)

numericalX = X.select_dtypes(np.number).reset_index().drop(['index'],axis=1)
categoricalX = X.select_dtypes(object)

# OneHot encoding
encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)
X = pd.concat([numericalX, encoded_categorical], axis = 1)

# test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [7]:
regr = DecisionTreeRegressor(max_depth=5)
model = regr.fit(X_train, y_train)

In [8]:
print("test data accuracy was: ",regr.score(X_test, y_test))
print("train data accuracy was: ",regr.score(X_train, y_train))

test data accuracy was:  0.1636746967276509
train data accuracy was:  0.6656532021942065


In [9]:
# Doesn't look too good on accuracy, trying with less than 5
regr = DecisionTreeRegressor(max_depth=4)
model = regr.fit(X_train, y_train)
print("test data accuracy was: ",regr.score(X_test, y_test))
print("train data accuracy was: ",regr.score(X_train, y_train))

test data accuracy was:  0.19012704696940286
train data accuracy was:  0.619680026984313


In [10]:
# not really satisfied on the test data, even after max_depth adjustment. No idea if or what I am doing wrong

In [11]:
from sklearn.tree import export_text

r = export_text(regr, feature_names= list(X.columns))
print(r)

|--- LASTGIFT <= 20.25
|   |--- LASTGIFT <= 13.05
|   |   |--- AVGGIFT <= 6.32
|   |   |   |--- LASTGIFT <= 7.50
|   |   |   |   |--- value: [5.67]
|   |   |   |--- LASTGIFT >  7.50
|   |   |   |   |--- value: [9.07]
|   |   |--- AVGGIFT >  6.32
|   |   |   |--- AVGGIFT <= 10.45
|   |   |   |   |--- value: [10.37]
|   |   |   |--- AVGGIFT >  10.45
|   |   |   |   |--- value: [14.41]
|   |--- LASTGIFT >  13.05
|   |   |--- LASTGIFT <= 15.38
|   |   |   |--- AVGGIFT <= 11.24
|   |   |   |   |--- value: [14.59]
|   |   |   |--- AVGGIFT >  11.24
|   |   |   |   |--- value: [16.56]
|   |   |--- LASTGIFT >  15.38
|   |   |   |--- LFC2 <= 3.00
|   |   |   |   |--- value: [50.00]
|   |   |   |--- LFC2 >  3.00
|   |   |   |   |--- value: [18.91]
|--- LASTGIFT >  20.25
|   |--- AVGGIFT <= 40.04
|   |   |--- CONTROLN <= 191663.00
|   |   |   |--- LASTGIFT <= 28.50
|   |   |   |   |--- value: [25.36]
|   |   |   |--- LASTGIFT >  28.50
|   |   |   |   |--- value: [36.95]
|   |   |--- CONTROLN >  19

In [12]:
# different models

In [13]:
# scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(numericalX)
numericalX_std = pd.DataFrame(scaler.transform(numericalX))

In [14]:
# refresh data
X2 = pd.concat([numericalX_std, encoded_categorical], axis = 1)
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state=12)

In [15]:
# Linear regression
LR = linear_model.LinearRegression()

In [16]:
# KNN
KNN = KNeighborsRegressor(n_neighbors=5)

In [17]:
# before
scores = []
models = [LR, KNN, regr] # insert model acronym here
for x in models:
    x.fit(X_train,y_train)
    scores.append(x.score(X_test, y_test)) 
    
d = {'model':models,'score':scores}
results = pd.DataFrame(d)
results

,model,score
0,LinearRegression(),0.135762
1,KNeighborsRegressor(),-0.200917
2,DecisionTreeRegressor(max_depth=4),0.190127


In [18]:
# after
scores = []
models = [LR, KNN, regr] # insert model acronym here
for x in models:
    x.fit(X2_train,y_train)
    scores.append(x.score(X2_test, y_test)) 

d2 = {'model':models,'score':scores}
results = pd.DataFrame(d2)
results

,model,score
0,LinearRegression(),0.135762
1,KNeighborsRegressor(),0.149697
2,DecisionTreeRegressor(max_depth=4),0.190127


In [19]:
# Conclusion model comparison: 
# DecisionTreeRegressor gives the highest scores 

In [20]:
# Random forest instead of model comparison

In [21]:
# refresh data
X3 = pd.concat([numericalX_std, encoded_categorical], axis = 1)
X3_train, X3_test, y_train, y_test = train_test_split(X3, y, test_size=0.2, random_state=12)

In [22]:
# applying Random forest classifier
clf = RandomForestRegressor()
RFR = clf.fit(X3_train, y_train)
pred = RFR.predict(X3_test)

r2 = r2_score(y_test,pred)
mse = mean_squared_error(y_test,pred)
mae = mean_absolute_error(y_test,pred)
rmse = sqrt(mse)

print('R2 Score:', r2)
print('Mean Squared Error:', mse)
print('Mean aboslute Error:', mae)
print('Root Mean Squared Error:',rmse)

R2 Score: 0.4111740483078671
Mean Squared Error: 85.69638406214654
Mean aboslute Error: 4.469743653250775
Root Mean Squared Error: 9.257234147527356


In [23]:
# Evaluate the result of your model and estimate how much better the result are for the bussiness 
# in comparison with the naive scenario we discuss on Monday.
# I really don't understand this unclear instruction, but I would say that applying 
# decision trees and base the campaign on that will most likely give highest return on investment